<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
from scipy.stats import pearsonr
import math

# Import dataset

In [ ]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

In [ ]:
dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)

In [ ]:
dataset.shape

(1867777, 4)

# Data processing

In [ ]:
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [ ]:
Counter(Y)

Counter({0.0: 27522,
         1.0: 10098,
         2.0: 105370,
         3.0: 417792,
         4.0: 1081772,
         5.0: 22287,
         6.0: 116199,
         7.0: 6223,
         8.0: 9161,
         9.0: 8683,
         10.0: 7871,
         11.0: 54799})

In [ ]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter = np.zeros((X.shape[0], X.shape[1]))

for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
            high_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]
            high_pass_filter[i][j] = X[i][j] - low_pass_filter[i][j]

## Frame preparation

In [ ]:
Fs = 4
frame_size = Fs*2 # 8 # 2 seconds of data
hop_size =  Fs*1

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

In [ ]:
Counter(row_label)

Counter({0.0: 6891,
         1.0: 2528,
         2.0: 26346,
         3.0: 104480,
         4.0: 270422,
         5.0: 5568,
         6.0: 29030,
         7.0: 1556,
         8.0: 2294,
         9.0: 2168,
         10.0: 1968,
         11.0: 13692})

## Checking ambiguity

In [ ]:
for i in range(row_frame.shape[0]):
  for j in range(row_frame.shape[0]):
    if (i != j) and np.array_equal(row_frame[i],row_frame[j]): #and (row_label[i] != row_label[j]):
      print("i = ",i, "label = ", row_label[i], "j = ",j,"label = ",row_label[j])

## Feature calculation

In [ ]:
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

In [ ]:
rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0


In [ ]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

In [ ]:
avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])
    
    value = value / (frame_size / polling_rate)
    avc.append(value)

In [ ]:
max_min = []  # max length - min length

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))

In [ ]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))

In [ ]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2] 
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+11] = angle[i][j]


In [ ]:
label = row_label

# Feature scaling

In [ ]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Train test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, label, test_size = 0.2, random_state = 0, stratify = label)

In [ ]:
Counter(y_train)

Counter({0.0: 5513,
         1.0: 2022,
         2.0: 21077,
         3.0: 83584,
         4.0: 216337,
         5.0: 4454,
         6.0: 23224,
         7.0: 1245,
         8.0: 1835,
         9.0: 1734,
         10.0: 1575,
         11.0: 10954})

## Checking ambiguity

In [ ]:
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    if (i != j) and np.array_equal(X_train[i],X_train[j]):
      print("i = ",i, "label = ", y_train[i], "j = ",j,"label = ",y_train[j])


# Algorithm

## Random forest

### Random oversampling

In [ ]:
sampling_dict = {0:6202, 1:6000, 2:23711, 3:94032, 4:243379, 5:6000, 7:6000, 8:6000, 9:6000, 10:6000, 11:12323}

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
Counter(y_train)

Counter({0.0: 243379,
         1.0: 243379,
         2.0: 243379,
         3.0: 243379,
         4.0: 243379,
         5.0: 243379,
         6.0: 243379,
         7.0: 243379,
         8.0: 243379,
         9.0: 243379,
         10.0: 243379,
         11.0: 243379})

### Adaptive synthetic sampling

In [ ]:
from imblearn.over_sampling import ADASYN
X_train, y_train = ADASYN(n_neighbors=3).fit_resample(X_train, y_train)

### Near Miss

In [ ]:
from imblearn.under_sampling import NearMiss
sampling_dict_nearmiss = {0.0: 5513, 1.0: 2022, 2.0: 21077, 3.0: 83584, 4.0: 10337, 5.0: 4454, 6.0: 23224, 7.0: 1245, 8.0: 1835, 9.0: 1734, 10.0: 1575, 11.0: 10954}
nm1 = NearMiss(version=1, sampling_strategy=sampling_dict_nearmiss)
X_train, y_train = nm1.fit_resample(X_train, y_train)

In [ ]:
Counter(y_train)

Counter({0.0: 5513,
         1.0: 2022,
         2.0: 21077,
         3.0: 83584,
         4.0: 10337,
         5.0: 4454,
         6.0: 23224,
         7.0: 1245,
         8.0: 1835,
         9.0: 1734,
         10.0: 1575,
         11.0: 10954})

## Random forest algorithm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = classifier.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,208,11,45,399,680,3,21,1,7,2,0,1
1,9,107,32,138,155,12,24,6,8,5,6,4
2,23,27,1243,882,3003,6,56,2,8,9,3,7
3,23,24,209,9633,10265,564,145,7,10,5,6,5
4,47,6,423,8431,45014,7,119,3,12,2,4,17
5,13,18,22,695,228,81,41,5,5,1,0,5
6,35,17,89,1473,2328,33,1792,6,4,8,8,13
7,8,13,28,68,90,7,16,46,14,4,11,6
8,6,19,10,113,162,6,16,11,80,8,13,15
9,1,15,16,107,169,4,7,7,16,65,17,10


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
accuracy

0.6353532000556811

In [ ]:
per_class_accuracy = []
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy.append(accuracy)

In [ ]:
per_class_accuracy

[0.11509146341463415,
 0.015060240963855422,
 0.2716388616290481,
 0.4620007754943777,
 0.8214961594122231,
 0.008743169398907104,
 0.3330968428520752,
 0.00684931506849315,
 0.08518518518518518,
 0.03308823529411765,
 0.005434782608695652,
 0.42736842105263156]

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [ ]:
pd.DataFrame(cm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
per_class_accuracy = []
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy.append(accuracy)

## XGBoost

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = classifier.predict(X_val)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,30,3,32,248,1064,0,0,0,1,0,0,0
1,0,37,8,168,275,2,12,1,0,0,3,0
2,0,5,230,762,4267,1,3,1,0,0,0,0
3,3,43,67,12200,8552,4,6,1,1,0,19,0
4,7,2,134,7508,46421,1,7,1,2,0,1,1
5,0,9,7,628,448,17,3,1,1,0,0,0
6,29,9,21,1727,2866,6,1143,1,2,0,2,0
7,0,7,12,81,199,1,4,3,1,0,2,1
8,0,6,11,134,293,0,0,3,10,0,2,0
9,0,4,6,107,294,2,2,2,7,0,8,2


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
accuracy

0.6523466361134609

## Naive bayes

In [ ]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = classifier.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
accuracy

0.4682457248712375

## Neural Network

## Oversampling to balance the data

Different kinds of oversampling processes are performed below. Choose any one of them

### Random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

### Adaptive synthetic sampling methond

In [ ]:
from imblearn.over_sampling import ADASYN
X_train, y_train = ADASYN( n_neighbors=1).fit_resample(X_train, y_train)

In [ ]:
Counter(y_train)

Counter({0.0: 242346,
         1.0: 242263,
         2.0: 235673,
         3.0: 228456,
         4.0: 242549,
         5.0: 242030,
         6.0: 246094,
         7.0: 242529,
         8.0: 242240,
         9.0: 242504,
         10.0: 242628,
         11.0: 245037})

### Neural Network Algorithm

In [ ]:
class_weights = {}
for classes in np.unique(y_train):
    class_weights[classes] = len(X_train)/(np.count_nonzero(y_train == classes)
                                     *len(np.unique(y_train)))

In [ ]:
class_weights

{0.0: 5.646562670052603,
 1.0: 15.39540059347181,
 2.0: 1.4769416899938321,
 3.0: 0.37243371937212866,
 4.0: 0.14389355496285888,
 5.0: 6.989110911540188,
 6.0: 1.340402170168791,
 7.0: 25.003614457831326,
 8.0: 16.964305177111715,
 9.0: 17.95242214532872,
 10.0: 19.764761904761905,
 11.0: 2.841838597772503}

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size=1000, validation_data= (X_val, y_val))#, class_weight=class_weights, verbose=1)

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred[6,:], y_val[6]

(array([2.4257833e-02, 4.3300386e-03, 1.0035785e-01, 2.9813433e-01,
        4.0100390e-01, 2.3007175e-02, 1.0948785e-01, 3.2196267e-06,
        2.0172559e-04, 1.3925997e-03, 9.9036528e-04, 3.6833156e-02],
       dtype=float32), 3.0)

In [ ]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val,y_pred_)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,122,12,64,377,775,3,10,2,7,2,4,0
1,11,83,38,181,139,21,3,8,9,4,7,2
2,34,7,796,926,3374,9,49,9,16,5,12,32
3,23,16,304,9553,10693,26,208,9,26,9,15,14
4,38,7,417,4292,48964,11,251,14,34,8,6,43
5,9,10,20,698,256,61,36,4,12,3,1,4
6,49,26,77,1694,2385,10,1519,3,14,2,8,19
7,7,6,33,101,74,8,3,30,34,6,5,4
8,9,14,16,113,174,2,3,15,82,12,11,8
9,5,11,12,146,150,4,9,8,28,35,19,7


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_)

In [ ]:
accuracy

0.6657208022358093

In [ ]:
per_class_accuracy = []
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy.append(accuracy)

In [ ]:
per_class_accuracy

[0.08853410740203194,
 0.16403162055335968,
 0.15107230973619282,
 0.4571688361408882,
 0.9053157067578811,
 0.05475763016157989,
 0.2616259042369962,
 0.09646302250803858,
 0.1786492374727669,
 0.08064516129032258,
 0.17302798982188294,
 0.3133674214755296]

### Focal loss

In [ ]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [ ]:
model.compile(loss=focal_loss(gamma=1,alpha=1), optimizer=Adam(learning_rate = 0.0001), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size=1000, validation_data= (X_val, y_val))

Epoch 1/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9861 - accuracy: 0.1398 - val_loss: 42.9836 - val_accuracy: 0.0046
Epoch 2/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9858 - accuracy: 0.1248 - val_loss: 42.9839 - val_accuracy: 0.0564
Epoch 3/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9858 - accuracy: 0.1227 - val_loss: 42.9843 - val_accuracy: 0.0119
Epoch 4/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9857 - accuracy: 0.0566 - val_loss: 42.9836 - val_accuracy: 0.0042
Epoch 5/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9859 - accuracy: 0.0816 - val_loss: 42.9842 - val_accuracy: 0.0046
Epoch 6/100
374/374 [==============================] - 5s 14ms/step - loss: 42.9858 - accuracy: 0.1139 - val_loss: 42.9841 - val_accuracy: 0.0293
Epoch 7/100
241/374 [==================>...........] - ETA: 1s - loss: 43.0538 - accuracy: 0.0825

KeyboardInterrupt: ignored

## CNN

In [ ]:
X_train = X_train.reshape(-1, input_vector.shape[1], 1, 1)
X_val = X_val.reshape(-1, input_vector.shape[1], 1, 1)

In [ ]:
model = Sequential()

model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))
model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))
#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

## CNN model using data augmentation

In [6]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)


X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

low_pass_filter_alpha8 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha8 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha6 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha6 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha7 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha7 = np.zeros((X.shape[0], X.shape[1]))

alpha = 0.8
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter_alpha8[i][j] = X[i][j]
            high_pass_filter_alpha8[i][j] = X[i][j]
        else:
            low_pass_filter_alpha8[i][j] = alpha*X[i-1][j] + (1-alpha)*X[i][j]
            high_pass_filter_alpha8[i][j] = X[i][j] - low_pass_filter_alpha8[i][j]

alpha = 0.6
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter_alpha6[i][j] = X[i][j]
            high_pass_filter_alpha6[i][j] = X[i][j]
        else:
            low_pass_filter_alpha6[i][j] = alpha*X[i-1][j] + (1-alpha)*X[i][j]
            high_pass_filter_alpha6[i][j] = X[i][j] - low_pass_filter_alpha6[i][j]

alpha = 0.7
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter_alpha7[i][j] = X[i][j]
            high_pass_filter_alpha7[i][j] = X[i][j]
        else:
            low_pass_filter_alpha7[i][j] = alpha*X[i-1][j] + (1-alpha)*X[i][j]
            high_pass_filter_alpha7[i][j] = X[i][j] - low_pass_filter_alpha7[i][j]

# frame preparation
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame_alpha8, low_pass_label_alpha8 = get_frames(low_pass_filter_alpha8, frame_size, hop_size, Y)
high_pass_frame_alpha8, high_pass_label_alpha8 = get_frames(high_pass_filter_alpha8, frame_size, hop_size, Y)
low_pass_frame_alpha7, low_pass_label_alpha7 = get_frames(low_pass_filter_alpha7, frame_size, hop_size, Y)
high_pass_frame_alpha7, high_pass_label_alpha7 = get_frames(high_pass_filter_alpha7, frame_size, hop_size, Y)
low_pass_frame_alpha6, low_pass_label_alpha6 = get_frames(low_pass_filter_alpha6, frame_size, hop_size, Y)
high_pass_frame_alpha6, high_pass_label_alpha6 = get_frames(high_pass_filter_alpha6, frame_size, hop_size, Y)

low_pass_frame = np.concatenate((low_pass_frame_alpha8,low_pass_frame_alpha7,low_pass_frame_alpha6))

high_pass_frame = np.concatenate((high_pass_frame_alpha8,high_pass_frame_alpha7,high_pass_frame_alpha6))

low_pass_label = np.concatenate((low_pass_label_alpha8,low_pass_label_alpha7,low_pass_label_alpha6))

high_pass_label = np.concatenate((high_pass_label_alpha8,high_pass_label_alpha7,high_pass_label_alpha6))

X_train, X_val, y_train, y_val = train_test_split(row_frame, row_label, test_size = 0.2, random_state = 0, stratify = row_label)

# Data Augmentation
low_pass_frame_6 = low_pass_frame[np.where(low_pass_label==6)]
X_train = np.concatenate((X_train,low_pass_frame_6))
low_pass_frame_0 = low_pass_frame[np.where(low_pass_label==0)]
X_train = np.concatenate((X_train,low_pass_frame_0))
low_pass_frame_1 = low_pass_frame[np.where(low_pass_label==1)]
X_train = np.concatenate((X_train,low_pass_frame_1))
low_pass_frame_7 = low_pass_frame[np.where(low_pass_label==7)]
X_train = np.concatenate((X_train,low_pass_frame_7))
low_pass_frame_8 = low_pass_frame[np.where(low_pass_label==8)]
X_train = np.concatenate((X_train,low_pass_frame_8))
low_pass_frame_5 = low_pass_frame[np.where(low_pass_label==5)]
X_train = np.concatenate((X_train,low_pass_frame_5))
low_pass_frame_9 = low_pass_frame[np.where(low_pass_label==9)]
X_train = np.concatenate((X_train,low_pass_frame_9))
low_pass_frame_10 = low_pass_frame[np.where(low_pass_label==10)]
X_train = np.concatenate((X_train,low_pass_frame_10))

low_pass_label_6 = low_pass_label[np.where(low_pass_label==6)]
y_train = np.concatenate((y_train,low_pass_label_6))
low_pass_label_0 = low_pass_label[np.where(low_pass_label==0)]
y_train = np.concatenate((y_train,low_pass_label_0))
low_pass_label_1 = low_pass_label[np.where(low_pass_label==1)]
y_train = np.concatenate((y_train,low_pass_label_1))
low_pass_label_7 = low_pass_label[np.where(low_pass_label==7)]
y_train = np.concatenate((y_train,low_pass_label_7))
low_pass_label_8 = low_pass_label[np.where(low_pass_label==8)]
y_train = np.concatenate((y_train,low_pass_label_8))
low_pass_label_5 = low_pass_label[np.where(low_pass_label==5)]
y_train = np.concatenate((y_train,low_pass_label_5))
low_pass_label_9 = low_pass_label[np.where(low_pass_label==9)]
y_train = np.concatenate((y_train,low_pass_label_9))
low_pass_label_10 = low_pass_label[np.where(low_pass_label==10)]
y_train = np.concatenate((y_train,low_pass_label_10))

high_pass_frame_6 = high_pass_frame[np.where(high_pass_label==6)]
X_train = np.concatenate((X_train,high_pass_frame_6))
high_pass_frame_0 = high_pass_frame[np.where(high_pass_label==0)]
X_train = np.concatenate((X_train,high_pass_frame_0))
high_pass_frame_1 = high_pass_frame[np.where(high_pass_label==1)]
X_train = np.concatenate((X_train,high_pass_frame_1))
high_pass_frame_7 = high_pass_frame[np.where(high_pass_label==7)]
X_train = np.concatenate((X_train,high_pass_frame_7))
high_pass_frame_8 = high_pass_frame[np.where(high_pass_label==8)]
X_train = np.concatenate((X_train,high_pass_frame_8))
high_pass_frame_5 = high_pass_frame[np.where(high_pass_label==5)]
X_train = np.concatenate((X_train,high_pass_frame_5))
high_pass_frame_9 = high_pass_frame[np.where(high_pass_label==9)]
X_train = np.concatenate((X_train,high_pass_frame_9))
high_pass_frame_10 = high_pass_frame[np.where(high_pass_label==10)]
X_train = np.concatenate((X_train,high_pass_frame_10))

high_pass_label_6 = high_pass_label[np.where(high_pass_label==6)]
y_train = np.concatenate((y_train,high_pass_label_6))
high_pass_label_0 = high_pass_label[np.where(high_pass_label==0)]
y_train = np.concatenate((y_train,high_pass_label_0))
high_pass_label_1 = high_pass_label[np.where(high_pass_label==1)]
y_train = np.concatenate((y_train,high_pass_label_1))
high_pass_label_7 = high_pass_label[np.where(high_pass_label==7)]
y_train = np.concatenate((y_train,high_pass_label_7))
high_pass_label_8 = high_pass_label[np.where(high_pass_label==8)]
y_train = np.concatenate((y_train,high_pass_label_8))
high_pass_label_5 = high_pass_label[np.where(high_pass_label==5)]
y_train = np.concatenate((y_train,high_pass_label_5))
high_pass_label_9 = high_pass_label[np.where(high_pass_label==9)]
y_train = np.concatenate((y_train,high_pass_label_9))
high_pass_label_10 = high_pass_label[np.where(high_pass_label==10)]
y_train = np.concatenate((y_train,high_pass_label_10))

In [ ]:
X_train = X_train.reshape(-1, frame_size, 3, 1)
X_val = X_val.reshape(-1, frame_size, 3, 1)

model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))

#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))

model.add(Flatten())

#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
2142/2142 [==============================] - 39s 18ms/step - loss: 1.4456 - accuracy: 0.4944 - val_loss: 1.1105 - val_accuracy: 0.5307
Epoch 2/50
2142/2142 [==============================] - 38s 18ms/step - loss: 1.3088 - accuracy: 0.5165 - val_loss: 1.0654 - val_accuracy: 0.5327
Epoch 3/50
2142/2142 [==============================] - 40s 19ms/step - loss: 1.2616 - accuracy: 0.5256 - val_loss: 1.0287 - val_accuracy: 0.5571
Epoch 4/50
2142/2142 [==============================] - 40s 19ms/step - loss: 1.2221 - accuracy: 0.5401 - val_loss: 1.0458 - val_accuracy: 0.5327
Epoch 5/50
2142/2142 [==============================] - 38s 18ms/step - loss: 1.1868 - accuracy: 0.5553 - val_loss: 1.0569 - val_accuracy: 0.5335
Epoch 6/50
2142/2142 [==============================] - 39s 18ms/step - loss: 1.1540 - accuracy: 0.5661 - val_loss: 1.0150 - val_accuracy: 0.5483
Epoch 7/50
2142/2142 [==============================] - 39s 18ms/step - loss: 1.1235 - accuracy: 0.5780 - val_loss: 1.0003 -